# 문화/게임 콘텐츠 용어 말뭉치 - 데이터 전처리

## 프로젝트 개요
- **목표**: 문화/게임 분야 용어 정의 생성 모델 파인튜닝
- **데이터**: AI Hub 문화, 게임 콘텐츠 분야 용어 말뭉치
- **샘플링**: 5만 문장 (권장사항 준수)

---

## 1. 환경 설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import pandas as pd
import random
from pathlib import Path

# 랜덤 시드 고정 (재현성)
random.seed(42)

DATA_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/data/')
OUTPUT_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/processed/')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print("환경 설정 완료")

환경 설정 완료


## 2. 데이터 로드

In [3]:
def load_json(file_path):
    """JSON 파일 로드"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 용어 사전 로드
terms = load_json(DATA_PATH / '용어.json')
print(f"용어 수: {len(terms):,}개")

# 용례 데이터 로드
examples_game = load_json(DATA_PATH / '용례_게임.json')
examples_leisure = load_json(DATA_PATH / '용례_레저.json')
examples_media = load_json(DATA_PATH / '용례_미디어.json')

print(f"\n용례 문장 수:")
print(f"  - 게임: {len(examples_game):,}개")
print(f"  - 레저: {len(examples_leisure):,}개")
print(f"  - 미디어: {len(examples_media):,}개")
print(f"  - 총합: {len(examples_game) + len(examples_leisure) + len(examples_media):,}개")

용어 수: 90,433개

용례 문장 수:
  - 게임: 199,504개
  - 레저: 75,384개
  - 미디어: 67,736개
  - 총합: 342,624개


## 3. 용어 사전 DataFrame 생성

In [4]:
# 용어 데이터를 DataFrame으로 변환
terms_df = pd.DataFrame(terms)

# 주요 컬럼만 선택
terms_df = terms_df[['id', 'term', 'definition', 'pos', 'facet', 'top_level_domain', 'level2', 'level3']]

print(f"용어 DataFrame 생성 완료: {len(terms_df):,}개")
print(f"\n컬럼: {list(terms_df.columns)}")
print(f"\n도메인 분포:")
print(terms_df['top_level_domain'].value_counts())

terms_df.head()

용어 DataFrame 생성 완료: 90,433개

컬럼: ['id', 'term', 'definition', 'pos', 'facet', 'top_level_domain', 'level2', 'level3']

도메인 분포:
top_level_domain
게임     49348
미디어    20905
레저     20180
Name: count, dtype: int64


,id,term,definition,pos,facet,top_level_domain,level2,level3
0,162960,라이아,"리니지M 게임에 등장하는, 마령군왕의 집무실에 나오는 보스 몬스터인 '마령군왕 라이...",일반명사(NNG),몬스터,게임,MMORPG,리니지M
1,104112,가디언,리니지M 게임에 등장하는 에스카로스에서 출현하여 불 약점을 가진 몬스터.,일반명사(NNG),몬스터,게임,MMORPG,리니지M
2,213915,빛나는 방어의 반지,"리니지M 게임에서, 은으로 만들어진 반지로 마법 방어력을 올려 주는 아이템.",준명사(NPH),아이템,게임,MMORPG,리니지M
3,236682,쉐도우 스턴,"리니지M 게임에서, 흑정령 마법으로 자신을 그림자에 숨겨 2~6초 동안 대상의 위치...",준명사(NPH),기술,게임,MMORPG,리니지M
4,237567,스냅퍼의 방어 반지,"리니지M 게임에서, 장신구에 속하며 마법 방어력을 상승시켜 주는 아이템.",준명사(NPH),아이템,게임,MMORPG,리니지M


## 4. 학습 데이터 형식 변환

### 태스크: 용어 정의 생성
- **입력**: 용어 + 분야 정보
- **출력**: 용어의 정의

In [5]:
def create_instruction_data(row):
    """
    Instruction 형식의 학습 데이터 생성
    """
    term = row['term']
    definition = row['definition']
    domain = row['top_level_domain']
    facet = row['facet'] if pd.notna(row['facet']) else ''

    # Instruction 형식
    if facet:
        instruction = f"다음은 {domain} 분야의 {facet} 관련 용어입니다. '{term}'의 정의를 설명해주세요."
    else:
        instruction = f"다음은 {domain} 분야의 용어입니다. '{term}'의 정의를 설명해주세요."

    return {
        'instruction': instruction,
        'input': '',
        'output': definition,
        'term': term,
        'domain': domain,
        'facet': facet
    }

# 학습 데이터 생성
instruction_data = [create_instruction_data(row) for _, row in terms_df.iterrows()]

print(f"Instruction 데이터 생성 완료: {len(instruction_data):,}개")
print(f"\n예시:")
print(json.dumps(instruction_data[0], ensure_ascii=False, indent=2))

Instruction 데이터 생성 완료: 90,433개

예시:
{
  "instruction": "다음은 게임 분야의 몬스터 관련 용어입니다. '라이아'의 정의를 설명해주세요.",
  "input": "",
  "output": "리니지M 게임에 등장하는, 마령군왕의 집무실에 나오는 보스 몬스터인 '마령군왕 라이아'를 줄여 이르는 말.",
  "term": "라이아",
  "domain": "게임",
  "facet": "몬스터"
}


## 5. 5만 문장 샘플링

In [6]:
# 5만개 샘플링 (용어 데이터가 5만개 미만이면 전체 사용)
SAMPLE_SIZE = 50000

if len(instruction_data) > SAMPLE_SIZE:
    sampled_data = random.sample(instruction_data, SAMPLE_SIZE)
    print(f"샘플링 완료: {len(sampled_data):,}개 (원본: {len(instruction_data):,}개)")
else:
    sampled_data = instruction_data
    print(f"전체 데이터 사용: {len(sampled_data):,}개")

# 도메인별 분포 확인
domain_counts = {}
for item in sampled_data:
    domain = item['domain']
    domain_counts[domain] = domain_counts.get(domain, 0) + 1

print(f"\n샘플링 후 도메인 분포:")
for domain, count in sorted(domain_counts.items(), key=lambda x: -x[1]):
    print(f"  - {domain}: {count:,}개 ({count/len(sampled_data)*100:.1f}%)")

샘플링 완료: 50,000개 (원본: 90,433개)

샘플링 후 도메인 분포:
  - 게임: 27,327개 (54.7%)
  - 미디어: 11,597개 (23.2%)
  - 레저: 11,076개 (22.2%)


## 6. Train/Validation 분할

In [7]:
from sklearn.model_selection import train_test_split

# 90% train, 10% validation 분할
train_data, val_data = train_test_split(sampled_data, test_size=0.1, random_state=42)

print(f"학습 데이터: {len(train_data):,}개")
print(f"검증 데이터: {len(val_data):,}개")

학습 데이터: 45,000개
검증 데이터: 5,000개


## 7. 데이터 저장

In [8]:
def save_jsonl(data, file_path):
    """JSONL 형식으로 저장 (각 줄이 하나의 JSON 객체)"""
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# JSONL 형식으로 저장
save_jsonl(train_data, OUTPUT_PATH / 'train.jsonl')
save_jsonl(val_data, OUTPUT_PATH / 'val.jsonl')

# JSON 형식으로도 저장
with open(OUTPUT_PATH / 'train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open(OUTPUT_PATH / 'val.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print(f"데이터 저장 완료!")
print(f"  - {OUTPUT_PATH / 'train.jsonl'}")
print(f"  - {OUTPUT_PATH / 'val.jsonl'}")
print(f"  - {OUTPUT_PATH / 'train.json'}")
print(f"  - {OUTPUT_PATH / 'val.json'}")

데이터 저장 완료!
  - /content/drive/MyDrive/LLM_FineTuning_Project/processed/train.jsonl
  - /content/drive/MyDrive/LLM_FineTuning_Project/processed/val.jsonl
  - /content/drive/MyDrive/LLM_FineTuning_Project/processed/train.json
  - /content/drive/MyDrive/LLM_FineTuning_Project/processed/val.json


## 8. 저장된 데이터 확인

In [9]:
# 저장된 데이터 샘플 확인
print("=" * 60)
print("학습 데이터 샘플 (3개)")
print("=" * 60)

for i, item in enumerate(train_data[:3]):
    print(f"\n[샘플 {i+1}]")
    print(f"Instruction: {item['instruction']}")
    print(f"Output: {item['output'][:100]}..." if len(item['output']) > 100 else f"Output: {item['output']}")
    print("-" * 40)

학습 데이터 샘플 (3개)

[샘플 1]
Instruction: 다음은 미디어 분야의 추상물 관련 용어입니다. '차점작'의 정의를 설명해주세요.
Output: 최고점이나 기준점에 다음가는 점수를 받은 작품.
----------------------------------------

[샘플 2]
Instruction: 다음은 미디어 분야의 추상물 관련 용어입니다. '총생산량'의 정의를 설명해주세요.
Output: 일정한 기간 동안 생산된 모든 재화의 수량.
----------------------------------------

[샘플 3]
Instruction: 다음은 게임 분야의 온라인 관련 용어입니다. '하이잘산'의 정의를 설명해주세요.
Output: 월드오브워크래프트 게임에서, 아즈샤라로 둘러싸여 있는 칼림도어에서 가장 높은 산 지역.
----------------------------------------
